# WESTERN NORTH PACIFIC BASIN DATA SELECTION

In [1]:
import xarray as xr
from matplotlib.pylab import *
import numpy as np
import pandas as pd
from geopy import distance
from mpl_toolkits.axes_grid1 import make_axes_locatable
from geographiclib.geodesic import Geodesic
import time

In [2]:
def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [3]:
xds_tracks = xr.open_dataset('Allstorms.ibtracs_wmo.v03r09.nc')

In [4]:
xds_tracks.basin.key

'0 = NA - North Atlantic\n1 = SA - South Atlantic\n2 = WP - West Pacific\n3 = EP - East Pacific\n4 = SP - South Pacific\n5 = NI - North Indian\n6 = SI - South Indian\n7 = AS - Arabian Sea\n8 = BB - Bay of Bengal\n9 = EA - Eastern Australia\n10 = WA - Western Australia\n11 = CP - Central Pacific\n12 = CS - Carribbean Sea\n13 = GM - Gulf of Mexico\n14 = MM - Missing'

Parece que WNP corresponde al valor 2 = West Pacific. Suponemos que sea la parte del norte ya que existe otras cuencas como Western Australia que corresponderá a la zona sur.

Con lo cual el primer filtrado de los ciclones viene por el atributo basin = 2.

In [5]:
xds_tracks = xds_tracks.sel(storm = np.unique(np.where(xds_tracks.basin == 2)[0])) #selecciono storm con Cuenca en WP
#falta seleccionar por fecha (ya solo tengo doses y nans)

Ahora como ejemplo sencillo, vamos a seleccionar los datos para la cuenca WP donde la presión del centro se encuentra entre 980-1000 mb y  latitud entre 5-10º.

In [25]:
xds_tracks

<xarray.Dataset>
Dimensions:          (center: 26, storm: 1719, time: 137)
Dimensions without coordinates: center, storm, time
Data variables:
    storm_sn         (storm) |S13 ...
    name             (storm) |S57 ...
    numObs           (storm) int16 ...
    season           (storm) int16 ...
    track_type       (storm) int8 ...
    genesis_basin    (storm) int8 ...
    num_basins       (storm) int8 ...
    basin            (storm, time) float32 2.0 2.0 2.0 2.0 ... nan nan nan nan
    wind_avg_period  (center) float32 ...
    source           (center) |S10 ...
    time_wmo         (storm, time) datetime64[ns] ...
    lat_wmo          (storm, time) float32 ...
    lon_wmo          (storm, time) float32 ...
    alt              (storm, time) float32 ...
    wind_wmo         (storm, time) float32 ...
    pres_wmo         (storm, time) float32 1010.0 1010.0 1000.0 ... nan nan nan
    sub_basin        (storm, time) float32 ...
    nature_wmo       (storm, time) float32 ...
    source_wmo       (storm, time) float32 ...
    dist2land        (storm, time) float32 ...
    landfall         (storm, time) float32 ...
Attributes:
    Title:                IBTrACS-WMO: NetCDF reformat
    Version:              v03r09
    Description:          IBTrACS-WMO data reformatted to contain \nall data ...
    cdm_datatype:         Trajectory
    trajectoryDimension:  storm
    Conventions:          CF-1.0
    metadata_link:        gov.noaa.ncdc:C00834

In [ ]:
import datetime

d = datetime.date(1997, 12, 31)
np.datetime64(d)

In [189]:
lats = []
lons = []
pres = []
time = []
st_idx = []
for i in xds_tracks.storm.values:
    idx = np.argwhere(xds_tracks.sel(storm = i).time_wmo[~np.isnan(xds_tracks.sel(storm = i).time_wmo)].values > np.datetime64(d))
   #idx me selecciona los datos para fechas a partir de 1998
    if len(idx) == 0: #esta condicion significa que es un array vacio, con lo cual no hay fechas a partir de 1998
        continue
    else:
        
        lats.append(xds_tracks.sel(storm = i).lat_wmo.values)
        lons.append(xds_tracks.sel(storm = i).lon_wmo.values)
        pres.append(xds_tracks.sel(storm = i).pres_wmo.values)
        time.append(xds_tracks.sel(storm = i).time_wmo.values)
        st_idx.append(i)

Ahora tengo para cada tormenta de la cuenca, los valores de latitud, longitud, presión y tiempo correspondiente. Podemos guardarlo en un dataset.

In [195]:
WNP_basin = xr.Dataset({'latitud': (('storm','time'), np.array(lats)),
           'longitud':(('storm','time'), np.array(lons)),
           'presion': (('storm','time'), np.array(pres)),
           'fechas': (('storm','time'), np.array(time))},
          {'storm': np.array(st_idx),
          'time': xds_tracks.time.values})

In [196]:
WNP_basin

<xarray.Dataset>
Dimensions:   (storm: 418, time: 137)
Coordinates:
  * storm     (storm) int64 1301 1302 1303 1304 1305 ... 1715 1716 1717 1718
  * time      (time) int64 0 1 2 3 4 5 6 7 8 ... 129 130 131 132 133 134 135 136
Data variables:
    latitud   (storm, time) float32 20.8 21.699999 22.1 22.4 ... nan nan nan nan
    longitud  (storm, time) float32 118.799995 118.799995 118.899994 ... nan nan
    presion   (storm, time) float32 1006.0 1004.0 1004.0 1004.0 ... nan nan nan
    fechas    (storm, time) datetime64[ns] 1998-07-08T00:00:00.000043520 ... NaT

In [197]:
WNP_basin.to_netcdf('basinWNP.nc4')